In [1]:
import sys
sys.version

'3.8.4 (default, Sep 10 2020, 14:12:06) \n[Clang 11.0.3 (clang-1103.0.32.62)]'

### What does a context manager look like

In [ ]:
with open('myfile.txt', 'w') as f:
    f.write("Hello World!\n")

In [ ]:
f = open('myfile.txt', 'w')
try:
    f.write("Hello World!\n")
finally:
    f.close()

### [PEP 343 -- The "with" Statement](https://www.python.org/dev/peps/pep-0343/) (13-May-2005)

This PEP adds a new statement "with" to the Python language to make it possible to factor out standard uses of try/finally statements.

In this PEP, context managers provide `__enter__()` and `__exit__()` methods that are invoked on entry to and exit from the body of the with statement.

#### context management protocol

This PEP proposes that the protocol consisting of the `__enter__()` and `__exit__()` methods be known as the "context management protocol", and that objects that implement that protocol be known as "context managers".

```
with EXPR as VAR:
    BLOCK
```

```
mgr = (EXPR)
exit = type(mgr).__exit__  # Not calling it yet
value = type(mgr).__enter__(mgr)
exc = True
try:
    try:
        VAR = value  # Only if "as VAR" is present
        BLOCK
    except:
        # The exceptional case is handled here
        exc = False
        if not exit(mgr, *sys.exc_info()):
            raise
        # The exception is swallowed if exit() returns true
finally:
    # The normal and non-local-goto cases are handled here
    if exc:
        exit(mgr, None, None, None)
```

[Is Python *with* statement exactly equivalent to a try - (except) - finally block?](https://stackoverflow.com/a/26096914)

#### mocking
```
o.return_value.__enter__.return_value = mock.Mock()
o.return_value.__exit__.return_value = mock.Mock()
@mock.patch.object(o, '__enter__')
@mock.patch.object(o, '__exit__')
```

### Basic context manager

In [2]:
def Foo1():
    def __init__(self):
        pass
    
with Foo1():
    print(1)

AttributeError: __enter__

In [3]:
def foo1():
    pass
    
with foo1():
    print(1)

AttributeError: __enter__

In [4]:
class Foo2():
    def __init__(self):
        print('__init__ is called')
        self.var = 0
        
    def __enter__(self):
        print('__enter__ is called')
        return self # by default None will be returned and it's fine to use `as`
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        print('__exit__ is called')
        if exc_type:
            print(f'exc_type: {exc_type}')
            print(f'exc_value: {exc_value}')
            print(f'exc_traceback: {exc_traceback}')

In [5]:
my_foo2 = Foo2()
with my_foo2 as f2:
    print(f"actions between __enter and __exit__, {f2.var}")

__init__ is called
__enter__ is called
actions between __enter and __exit__, 0
__exit__ is called


In [6]:
with Foo2() as f2:
    print(f"actions between __enter and __exit__, {f2.var}")
    var2 = "bar"

print(f2.var)
print(var2)

__init__ is called
__enter__ is called
actions between __enter and __exit__, 0
__exit__ is called
0
bar


##### A with statement does not create a scope
```
import threading
with threading.Lock() as my_lock:
    hacking()

with my_lock:
    keep_hacking()
```

#### error raised in `__enter__`

In [7]:
class Foo3():
    def __init__(self):
        print('__init__ is called')
        self.var = 0
        
    def __enter__(self):
        print('__enter__ is called')
        raise ValueError("exception in __enter__")
        return self
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        print('__exit__ is called')
        if exc_type:
            print(f'exc_type: {exc_type}')
            print(f'exc_value: {exc_value}')
            print(f'exc_traceback: {exc_traceback}')

with Foo3():
    print(f"actions between __enter and __exit__")

__init__ is called
__enter__ is called


ValueError: exception in __enter__

#### error raised in `__exit__`

In [8]:
class Foo3():
    def __init__(self):
        print('__init__ is called')
        self.var = 0
        
    def __enter__(self):
        print('__enter__ is called')
        return self
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        raise ValueError("exception in __exit__")
        print('__exit__ is called')
        if exc_type:
            print(f'exc_type: {exc_type}')
            print(f'exc_value: {exc_value}')
            print(f'exc_traceback: {exc_traceback}')

with Foo3():
    print(f"actions between __enter and __exit__")

__init__ is called
__enter__ is called
actions between __enter and __exit__


ValueError: exception in __exit__

#### error after entered

In [9]:
class Foo3():
    def __init__(self):
        print('__init__ is called')
        self.var = 0
        
    def __enter__(self):
        print('__enter__ is called')
        return self
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        print('__exit__ is called')
        if exc_type:
            print(f'exc_type: {exc_type}')
            print(f'exc_value: {exc_value}')
            print(f'exc_traceback: {exc_traceback}')

with Foo3():
    raise ValueError("exception when performing action")
    print(f"actions between __enter and __exit__")

__init__ is called
__enter__ is called
__exit__ is called
exc_type: <class 'ValueError'>
exc_value: exception when performing action
exc_traceback: <traceback object at 0x108a13f80>


ValueError: exception when performing action

#### error after entered (within `with` body) and `__exit__` returns true

In [10]:
class Foo3():
    def __init__(self):
        print('__init__ is called')
        self.var = 0
        
    def __enter__(self):
        print('__enter__ is called')
        return self
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        print('__exit__ is called')
        if exc_type:
            print(f'exc_type: {exc_type}')
            print(f'exc_value: {exc_value}')
            print(f'exc_traceback: {exc_traceback}')
            # the exception is handled in __exit__
        return True

with Foo3():
    raise ValueError("exception when performing action")
    print(f"actions between __enter and __exit__")

__init__ is called
__enter__ is called
__exit__ is called
exc_type: <class 'ValueError'>
exc_value: exception when performing action
exc_traceback: <traceback object at 0x1080d7dc0>


### Use cases

- open/close
- lock/release
- change/reset
- enter/exit
- start/stop



- file
- thread.LockType
- threading.Lock
- threading.RLock
- threading.Condition
- threading.Semaphore
- threading.BoundedSemaphore

### from contextlib import contextmanager

In [11]:
from contextlib import contextmanager

@contextmanager
def tag(name):
    print("<%s>" % name)  # <-- __enter__
    yield
    print("</%s>" % name) # <-- __exit__


In [12]:
with tag("table"), tag("tr"):
    print("hello")

<table>
<tr>
hello
</tr>
</table>


In [14]:
with tag("table"):
    with tag("tr"):
        print("hello")

<table>
<tr>
hello
</tr>
</table>


In [15]:
@tag("table")
@tag("tr")
def with_tag():
    print("hello")

with_tag()

<table>
<tr>
hello
</tr>
</table>


In [16]:
from contextlib import contextmanager

@contextmanager
def tag1(name):
    print("<%s>" % name)
    # yield
    print("</%s>" % name)

with tag1("table"), tag1("tr"):
    print("hello")

<table>
</table>


TypeError: 'NoneType' object is not an iterator

In [17]:
from contextlib import contextmanager

@contextmanager
def tag1(name):
    print("<%s>" % name)
    yield
    yield
    print("</%s>" % name)

with tag1("table"), tag1("tr"):
    print("hello")

<table>
<tr>
hello


RuntimeError: generator didn't stop

### [contextmanager](https://github.com/python/cpython/blob/v3.8.5/Lib/contextlib.py#L211-L241) (decorator)

- [_GeneratorContextManager(func, args, kwds)](https://github.com/python/cpython/blob/v3.8.5/Lib/contextlib.py#L97) (class)

https://github.com/python/cpython/blob/v3.8.5/Lib/test/test_contextlib.py

### try-except/try-finally

```
<setup>
try:
    <var> = <value>
    <do something>
except Exception:
    <handle the error>
finally:
    <cleanup>
```

```
@contextmanager
def cm1():
    <setup>
    try:
        yield <value>
    except Exception:
        <handle the error>
    finally:
        <cleanup>
with cm1() as <var>:
    <do something>
```

In [18]:
from __future__ import generator_stop
from contextlib import contextmanager
import weakref
import sys

@contextmanager
def tag(name):
    print("<%s>" % name)  # <-- __enter__
    yield
    print("</%s>" % name) # <-- __exit__

def genfunc():
    with tag("h1"):
        print("woops")
        yield

def run():
    gen = genfunc()
    ref = weakref.ref(gen, print)
    next(gen)

run()

# https://github.com/PyCQA/pylint/issues/2832
# [PEP 205 -- Weak References](https://www.python.org/dev/peps/pep-0205/)

<h1>
woops
<weakref at 0x1080d8f40; dead>


In [19]:
# from __future__ import generator_stop
from contextlib import contextmanager
import weakref
import sys

@contextmanager
def tag(name):
    print("<%s>" % name)  # <-- __enter__
    try:
        yield
    finally:
        print("</%s>" % name) # <-- __exit__

def genfunc():
    with tag("h1"):
        print("woops")
        yield

def run():
    gen = genfunc()
    ref = weakref.ref(gen, print)
    next(gen)

run()

<h1>
woops
<weakref at 0x1080d8a40; dead>
</h1>


### yield statement

[PEP 255 -- Simple Generators](https://www.python.org/dev/peps/pep-0255/) (18-May-2001)

[PEP 380 -- Syntax for Delegating to a Subgenerator](https://www.python.org/dev/peps/pep-0380/) (13-Feb-2009)

yield controls the flow of a generator function
- yield without value
